# Imports

In [0]:
import numpy as np
import sys

# Loading the features and vocabulary

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s content/gdrive/My\ Drive/NER /ner_dir

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/ner_dir/NER': Operation not supported


In [3]:
%cd ..
%cd /ner_dir


/
/content/gdrive/My Drive/NER


In [4]:
!ls

activations.py		  loss.py		README.md
Conll.ipynb		  NER_NN.ipynb		test_features.npy
dataset			  NER_NN_network.ipynb	train_features.npy
initial_experiment.ipynb  nn.py			val_features.npy
initialization.py	  __pycache__		vocab.npy


In [0]:
import activations
import loss
import initialization as init_layer

In [0]:
vocab = np.load('vocab.npy')
train_features = np.load('train_features.npy')
test_features = np.load('test_features.npy')
val_features = np.load('val_features.npy')

In [19]:
train_features.shape

(204566, 900)

In [0]:
C = 1
word_vector_dim = 300
num_tags = 10

# Neural Netwwork Architecture

In [0]:
nn_architecture = [
  {"layer_size": 900, "activation": "none"},
  {"layer_size": 300, "activation": "relu"},
  {"layer_size": 100, "activation": "relu"},
  {"layer_size":10, "activation": "softmax"}
]

# Initialize Parameters

In [0]:
def initialize_parameters(nn_architecture, initialization = "range_initialization", seed=5):

  parameters = {}
  num_layers = len(nn_architecture)

  for i in range(1, num_layers):

    if initialization == "range_initialization":
      parameters['W' + str(i)] = init_layer.range_initializtion(nn_architecture[i]["layer_size"],
                                            nn_architecture[i - 1]["layer_size"], seed)
    else:
      parameters['W' + str(i)] = init_layer.random_initializtion(nn_architecture[i]["layer_size"],
                                            nn_architecture[i - 1]["layer_size"], seed)

    parameters['b' + str(i)] = np.zeros((nn_architecture[i]["layer_size"], 1))

  return parameters

In [11]:
initialize_parameters(nn_architecture, initialization="range_initialization")

{'W1': array([[-0.0393161 ,  0.05242947, -0.04147617, ...,  0.01886445,
         -0.0673961 ,  0.03122635],
        [-0.06872783,  0.05610709,  0.01245434, ..., -0.02137381,
         -0.02523462,  0.06828883],
        [ 0.01937873, -0.02733413, -0.0442238 , ...,  0.0283923 ,
         -0.01530554, -0.06579863],
        ...,
        [ 0.00216721,  0.0385514 , -0.00354598, ...,  0.05043729,
         -0.00701171, -0.00209508],
        [-0.06960359, -0.06415798,  0.03159176, ...,  0.0246166 ,
         -0.02451427, -0.03227124],
        [ 0.01399195,  0.05700679,  0.00883558, ...,  0.01464168,
         -0.01428649,  0.02139781]]),
 'W2': array([[-0.06809749,  0.0908105 , -0.07183884, ..., -0.03299794,
          0.10121721, -0.01631657],
        [ 0.10426298,  0.11301164, -0.12085584, ..., -0.07972386,
         -0.11218951,  0.0330916 ],
        [-0.06441902,  0.04898989, -0.02037828, ...,  0.03267418,
         -0.11673346,  0.05408562],
        ...,
        [-0.08484582,  0.09470461,  0.0761

# Forward Propogation

In [0]:
# Z = W * X + b
# Here A is output of previous layer

def linear_forward(A_prev, W, b):
  return np.dot(W, A_prev) + b

# apply activation h:  A = h(X) 
def apply_activation(A, activation, alpha=0.01):

  if activation == "sigmoid":
    return activations.sigmoid(A)
  elif activation == "tanh":
    return activations.tanh(A)
  elif activation == "relu":
    return activations.relu(A)
  elif activation == "leaky_relu":
    return activations.leaky_relu(A, alpha)
  else:
    print(f"ERROR: {activation} activation not supported")
    sys.exit(1)
  
# driver forward propogation
def model_forward(X, parameters, nn_architecture, alpha=0.01):

  forward_cache = {}
  A = X
  num_layers = len(nn_architecture)

  for i in range(1, num_layers):
    A_prev = A
    W = parameters["W" + str(i)]
    b = parametes["b" + str(i)]

    Z = linear_forward(A_prev, W, b)
    A = apply_activation(Z, activation, alpha)

    forward_cache['Z' + str(i)] = Z
    forward_cache['A' + str(i - 1)] = A

  return A, forward_cache  

# Loss